In [ ]:
import cv2
#from imutils import paths
from tqdm import tqdm
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import tensorflow as tf
!pip install tflearn
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d,batch_normalization,avg_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected,flatten
from tflearn.layers.estimator import regression
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
trainfile=pd.read_csv('../input/ci-sc22-places-and-scene-recognition/train.csv') #share plz ci dataset in your drive from link
trainfile=np.array(trainfile)
x=trainfile[:,0] # namephoto
y=trainfile[:,1] # label
print(x[0])

In [ ]:
TRAIN_DIR='../input/ci-sc22-places-and-scene-recognition/train_images/train_images'
IMG_SIZE = 150
LR = 0.001
MODEL_NAME = 'cnn'
training_data = []

def create_train_data():
    #i=0
    for img in tqdm(os.listdir(TRAIN_DIR)):
        path = os.path.join(TRAIN_DIR, img)
        #print(path)
        imagename=path.split('/')[-1] #123.gpj
        #print(imagename)
        try:
           index=np.where(x==imagename)[0][0]
        except:
         continue
        s=np.array([0, 0, 0, 0, 0, 0])
        s[y[index]]=1
        #print(s)
        img_data = cv2.imread(path)
        img_data = cv2.resize(img_data, (IMG_SIZE, IMG_SIZE))
        training_data.append([np.array(img_data),s]) #x.gpg,[0,0,0,0,0,1]
        #i=i+1
        #if i == 3 :
         # break
    np.save('train_data.npy', training_data)
    return training_data

In [ ]:
if (os.path.exists('train_data.npy')): # If you have already created the dataset:
    train_data =np.load('train_data.npy',allow_pickle=True)
    print('exist')
else: # If dataset is not created:
    train_data = create_train_data()

In [ ]:
#plt.imshow(np.array(training_data[2][0]).reshape(50,50))
#plt.show()
print(len(train_data))

In [ ]:
train = train_data

X_train = np.array([i[0] for i in train]).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
y_train = [i[1] for i in train]
print(X_train.shape)

conv_input = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 3], name='input')
model = conv_2d(conv_input, 128, 3, activation='relu')
model = conv_2d(model, 128, 3, activation='relu')
model = conv_2d(model, 128, 3, activation='relu')
model = avg_pool_2d(model, 6)

model = conv_2d(model, 1024, 6, activation='relu')
model = avg_pool_2d(model, 6)

model = dropout(model, 0.25)

model = conv_2d(model, 1024, 6, activation='relu')
model = avg_pool_2d(model, 6)

convnet = dropout(model, 0.25)


model = flatten(convnet)
model = fully_connected(model, 128, activation='relu')
model = fully_connected(model, 256, activation='relu')
model = fully_connected(model, 6, activation='softmax')



model = regression(model, optimizer='adam', learning_rate=1e-3,loss='categorical_crossentropy', name='targets')
model = tflearn.DNN(model, tensorboard_dir='log')


if (os.path.exists('model.tfl.meta')):
    model.load('./model.tfl')
else:
    model.fit(X_train, y_train, n_epoch=25,validation_set=0.1, shuffle=True,
          show_metric=True, snapshot_step=500,
          snapshot_epoch=False, run_id=MODEL_NAME)

    model.save('model.tfl')

In [ ]:
TEST_DIR = '../input/ci-sc22-places-and-scene-recognition/test_images/test_images'
#import zipfile #plz upload test_images folder in colab as zip file
#from zipfile import *
#zip_ref = zipfile.ZipFile('test_images.zip', 'r') 
#zip_ref.extractall('/tmp') 
#zip_ref.close()
#TRAIN_DIR = '/tmp/test_images/'

imagename=[]
indexmax=[]
#for imgpath in tqdm(os.listdir(TRAIN_DIR)):
for img in tqdm(os.listdir(TEST_DIR)):
    path = os.path.join(TEST_DIR, img)
    img_data = cv2.imread(path)
    img_data = cv2.resize(img_data, (IMG_SIZE, IMG_SIZE))
    img_data = img_data.reshape(IMG_SIZE, IMG_SIZE, 3)
    x=path.split('/')[-1]
    try:        
        z=x.index("(")
        continue 
    except:
        imagename.append(x) 
        prediction = model.predict([img_data])[0]
        pred=np.array(prediction)
        maxval= max(prediction)
        indexmax.append(np.where(pred==maxval)[0][0])
     
     
        

my_submission = pd.DataFrame({'Image_name': imagename, 'Label': indexmax})
my_submission.to_csv('submission.csv', index=False)